In [170]:
import numpy as np
from scipy import stats

class Group:
    def __init__(self):
        # Each group has a unique ID and a dictionary to track interactions
        self.id = 0

        self.interactions = {}

class User:
    def __init__(self, alpha, beta):
        # Each user has a unique ID, preferences, and histories of group interactions
        self.id = 0

        self.groups = []

        self.interaction_history = []

        self.preferences = stats.beta(alpha, beta)

    def update_preferences(self):
        if not self.groups:
            # Handle the case when no groups are joined
            self.updated_preferences = np.array([])
            return

        total_size = sum([len(group.interactions) for group in self.groups])
        if total_size == 0:
            # Avoid division by zero if there are no interactions
            self.groups_cdf = np.array([1 / len(self.groups)] * len(self.groups))
        else:
            self.groups_cdf = np.array([len(group.interactions) / total_size for group in self.groups])

        # Perform convolution
        self.updated_preferences = np.convolve(self.preferences.pdf(np.linspace(0, 1, len(self.groups))), self.groups_cdf, mode='same')

        # Normalize the updated preferences to avoid sum being zero or NaN values
        if np.isnan(self.updated_preferences).any() or np.sum(self.updated_preferences) == 0:
            self.updated_preferences = np.array([1 / len(self.groups)] * len(self.groups))
        else:
            self.updated_preferences /= np.sum(self.updated_preferences)

    def join_group(self, group):
        
        self.groups.append(group)

        group.interactions[self] = 0


    def interact(self, group : Group):
        
        group.interactions[self] += 1

        self.interaction_history.append(group.id)


users = []
groups = []

user_growth_rate = 1
new_group_join_rate = 0.5
group_leave_rate = 0.25

new_group_threshold = 0.05
interaction_threshold = 0.95

num_timesteps = 100

alpha_hyperparameter = 10
beta_hyperparameter = 10 

initial_users = 10
initial_groups = 5

data_for_regression = []

for _ in range(initial_users):
    user = User(np.random.randint(1, alpha_hyperparameter), np.random.randint(1, beta_hyperparameter))
    users.append(user)
    user.id = len(users)

for _ in range(initial_groups):
    group = Group()
    groups.append(group)
    group.id = len(groups)

for user in users:
    for _ in range(int(stats.expon.rvs(new_group_join_rate))):
        group = np.random.choice(groups)
        user.join_group(group)
    
    user.update_preferences()


for _ in range(num_timesteps):

    for _ in range(int(stats.expon.rvs(user_growth_rate))): 
        
        user = User(np.random.randint(1, alpha_hyperparameter), np.random.randint(1, beta_hyperparameter))

        users.append(user)

        user.id = len(users)

    for user in users:
        
        for _ in range(int(stats.expon.rvs(new_group_join_rate))):
            
            if np.random.uniform() < new_group_threshold:

                group = Group()
                user.join_group(group)
                groups.append(group)
                group.id = len(groups)

            else:

                group = np.random.choice(groups)
                user.join_group(group)

        for _ in range(int(stats.expon.rvs(group_leave_rate))):

            if user.groups:

                group = np.random.choice(user.groups)
                user.groups.remove(group)

        if user.groups:
            user.update_preferences()
        else:
            user.updated_preferences = np.array([1])
    
        if np.random.uniform() < interaction_threshold and user.groups:
            
            group = np.random.choice(user.groups, p=user.updated_preferences)
            user.interact(group)
        
        else:
            user.interaction_history.append(0)

        current_group = user.interaction_history[i]
        previous_group = user.interaction_history[i-1]
        
        interactions_by_group = np.bincount(user.interaction_history)


        # user_id, num_groups, current_group, previous_group, interactions_by_group

        data_for_regression.append(user.id, len(interactions_by_group - 1), current_group, previous_group, interactions_by_group)




